In [1]:
from time import sleep

import requests
import json
import pandas as pd
import time
from collections import defaultdict
from datetime import datetime

from dotenv import load_dotenv
import os


In [2]:
load_dotenv(dotenv_path='app.env')

token = os.getenv('GITHUB_TOKEN')

In [3]:
# Read the CSV file
file_path = 'Data/Application_types.csv'
df = pd.read_csv(file_path)

# Convert the 'Keyword' column to a list
topic_list = df['Keyword'].tolist()

# Display the list
print(topic_list)


def write_json_data(fileName, data):
    with open(fileName, 'w') as file:
        json.dump(data, file, indent=4)


def append_json_data(fileName, data):
    # Try to read existing data if the file exists
    try:
        with open(fileName, 'r') as file:
            existing_data = json.load(file)
    except FileNotFoundError:
        existing_data = []  # Initialize as an empty list if the file doesn't exist

    # Append new data to existing data
    existing_data.append(data)

    # Write the updated data back to the file
    with open(fileName, 'w') as file:
        json.dump(existing_data, file, indent=4)


def sleep_if_rate_exhausted(response):
    core_limits = response.headers._store
    rate_left = core_limits['x-ratelimit-remaining'][1]

    if int(rate_left) > 0:
        return

    reset_time = core_limits['x-ratelimit-reset'][1]
    reset_timestamp = datetime.fromtimestamp(int(reset_time))
    current_time = datetime.now()
    sleep_duration = (reset_timestamp - current_time).total_seconds()
    print(f"Rate limit reached. Sleeping until reset at {reset_timestamp}")
    time.sleep(sleep_duration+5)  # Sleep until reset time + some delay time
    print("Rate limit reset. Resuming requests.")


['Dashboard', 'CMS', 'E-commerce', 'Portfolio', 'Blog', 'Chatbot', 'Social Network', 'Messaging', 'Forum', 'Community', 'News', 'Wiki', 'File Manager', 'Analytics', 'Project Management', 'Task Manager', 'Calendar', 'Notes', 'Kanban', 'ERP', 'CRM', 'POS', 'Inventory', 'To-do', 'Video Streaming', 'Music Player', 'Photo Gallery', 'Weather', 'Finance', 'Budget Tracker', 'Expense Tracker', 'Banking', 'Travel', 'Booking', 'Reservation', 'Hotel Management', 'Real Estate', 'Job Portal', 'Classroom', 'Learning Management System', 'Online Exam', 'Survey', 'Poll', 'E-learning', 'Quiz', 'Recipe', 'Cookbook', 'Portfolio', 'Resume Builder', 'Event Management', 'Gaming', 'Game Tracker', 'Game Leaderboard', 'VR', 'AR', '3D Modeling', 'Fitness', 'Workout Tracker', 'Health Tracker', 'Diet Planner', 'Meal Planner', 'Meditation', 'Habit Tracker', 'Productivity', 'Chat', 'Collaboration', 'Help Desk', 'Customer Support', 'Feedback', 'Documentation', 'Startup', 'Portfolio', 'Map', 'Geo-location', 'Travel Gui

In [4]:
def get_github_projects_of_topic(num_results,num_pages, topic):
    headers = {'Authorization': f'token {token}'}
    url = 'https://api.github.com/search/repositories'
    params = {
        'q': f'(language:JavaScript OR language:TypeScript) {topic} stars:>50',
        'sort': 'stars',
        'order': 'desc',
        'per_page': num_results,
        'page': num_pages
    }

    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        data = response.json()
        projects = [{'id': item['id'], 'name': item['name'], 'owner': item['owner']['login'],
                     'watchers': item['watchers_count'],
                     'created_at': item['created_at'], 'updated_at': item['updated_at']
                        , 'size': item['size'], 'stars': item['stargazers_count']
                        , 'open_issues_count': item['open_issues_count']
                        , 'forks': item['forks'],  'description': item['description'], 'topics': item['topics']
                     } for item in data['items']]
        
        
        
        for project in projects:
            if project['id'] in projects_set:
                continue
            unique_projects.append(project)
            projects_set.add(project['id'])
            if(len(unique_projects) >= total_num_results):
                break

        

        return unique_projects, response,len(unique_projects),len(projects)

    else:
        print(f"Error: {response.status_code}")
        return None, response

topic_dict = {}
num_topics = 40
projects_set = set()
for i in range(num_topics):
    currentTopic = topic_list[i]
    num_unique_results = 0
    num_pages=1
    unique_projects = []
    prev_results=0
    num_results = 10
    total_num_results = num_results
    while True:
        projects, response, num_unique_results, num_of_results_from_API = get_github_projects_of_topic(num_results=num_results, num_pages=num_pages,topic=currentTopic)
        
        if projects is not None:
            topic_dict[currentTopic] = projects
        
        if num_unique_results >= total_num_results:
            break
        
        if num_of_results_from_API == prev_results:
            break
        
        prev_results = num_of_results_from_API
        
        if num_results<100:
            num_results+=10
        else:
            num_pages+=1
        
        
        sleep_if_rate_exhausted(response)
        
    sleep_if_rate_exhausted(response)
    
        
    
        

print(topic_dict)








Rate limit reached. Sleeping until reset at 2024-11-16 23:05:30
Rate limit reset. Resuming requests.
Rate limit reached. Sleeping until reset at 2024-11-16 23:06:35
Rate limit reset. Resuming requests.
Rate limit reached. Sleeping until reset at 2024-11-16 23:07:40
Rate limit reset. Resuming requests.
{'Dashboard': [{'id': 88464704, 'name': 'vue-element-admin', 'owner': 'PanJiaChen', 'watchers': 88002, 'created_at': '2017-04-17T03:35:49Z', 'updated_at': '2024-11-16T13:06:02Z', 'size': 15433, 'stars': 88002, 'open_issues_count': 1355, 'forks': 30474, 'description': ':tada: A magical vue admin                                                                https://panjiachen.github.io/vue-element-admin', 'topics': ['admin', 'admin-dashboard', 'admin-template', 'axios', 'dashboard', 'desktop', 'element', 'element-ui', 'i18n', 'management-system', 'mock', 'tinymce', 'vue', 'vue-admin', 'vue-cli', 'vuex', 'webpack', 'webpack4', 'xlsx']}, {'id': 15111821, 'name': 'grafana', 'owner': 'grafana'

In [5]:
all_projects = []

filename = 'github_40_projects_with_topics_4'
for topic, projects in topic_dict.items():
    for project in projects:
        # Add the topic as a new field in each project dictionary
        project['topic'] = topic
        all_projects.append(project)

# Convert the list of all projects with topics into a DataFrame
df = pd.DataFrame(all_projects)

# Save the DataFrame to a CSV file
df.to_csv(f'{filename}.csv', index=False)

print(f"Data saved to {filename}.csv")

error_from = ""

Data saved to github_40_projects_with_topics_4.csv


In [6]:
def get_Sbom_of_projects(repo_df):
    headers = {'Authorization': f'token {token}'}
    url = 'https://api.github.com/repos/'
    owner = repo_df['owner']
    repo = repo_df['name']
    url = url + f'{owner}/{repo}/dependency-graph/sbom'

    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        data = response.json()
        # write_json_data('./sbom.json',data)
        packages = data['sbom']['packages']
        # Extract npm packages with name and version
        pkg_dict = defaultdict(dict)
        for package in packages:
            if package['externalRefs'][0]['referenceLocator'].startswith('pkg:npm'):
                try:
                    if len(package['versionInfo']) > 0:
                        if 'versionInfo' in package:
                            if package['name'].startswith('npm:'):
                                sbom_dict[repo][package['name'][4:]] = package['versionInfo']
                                pkg_dict[repo][package['name'][4:]] = package['versionInfo']
                            else:
                                sbom_dict[repo][package['name']] = package['versionInfo']
                                pkg_dict[repo][package['name']] = package['versionInfo']
                except:
                    # print(package)
                    pass
        # append_json_data('Data/40_Projects_1/append_dependencies_40.json', pkg_dict)

    else:
        print(f"Error: {response.status_code}")

    return response


sbom_dict = defaultdict(dict)
ctr = 0 
for topic, projects in topic_dict.items():
    for project in projects:
        response = get_Sbom_of_projects(project)
        sleep_if_rate_exhausted(response)
    print(f'{ctr}. {topic} done')
    ctr+=1

write_json_data('Data/40_With_Description/append_dependencies_40_4.json', sbom_dict)
# print(sbom_dict)

0. Dashboard done
1. CMS done
2. E-commerce done
3. Portfolio done
4. Blog done
5. Chatbot done
6. Social Network done
7. Messaging done
8. Forum done
9. Community done
10. News done
11. Wiki done
12. File Manager done
13. Analytics done
14. Project Management done
15. Task Manager done
16. Calendar done
17. Notes done
18. Kanban done
19. ERP done
20. CRM done
21. POS done
22. Inventory done
23. To-do done
24. Video Streaming done
25. Music Player done
26. Photo Gallery done
27. Weather done
28. Finance done
29. Budget Tracker done
30. Expense Tracker done
31. Banking done
32. Travel done
33. Booking done
34. Reservation done
35. Hotel Management done
36. Real Estate done
37. Job Portal done
38. Classroom done
39. Learning Management System done


In [5]:
x = json.load(open("Data/40_Projects/append_dependencies_40_1.json"))

print(x)

KeyboardInterrupt: 